In [0]:
# # Verify GPU
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize
# import psutil
# import humanize
# import os
# import GPUtil as GPU
# GPUs = GPU.getGPUs()
# # XXX: only one GPU on Colab and isn’t guaranteed
# gpu = GPUs[0]
# def printm():
#   process = psutil.Process(os.getpid())
#   print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
#   print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
# printm()

## Import used python libraries

In [0]:
# from google.colab import drive
# drive.mount('/content/gdrive/')
# %cd gdrive/My\ Drive/Nvidia/Lab3

In [0]:
# Downgrade Pillow to avoid errors
!pip install Pillow==3.4.2

import tensorflow as tf
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.models import Model
from keras import optimizers 
from keras import regularizers
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from time import time

import matplotlib.pyplot as plt
import numpy as np
np.random.seed(12)

from sklearn.metrics import classification_report, f1_score, accuracy_score
import glob
from tqdm import tqdm
import warnings

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
The folder you are executing pip from can no longer be found.


## Variable path Setting
- training, validation, testing and model path directories.

In [0]:
train_dir = "./dataset/lab3-1/7_flower/train"
val_dir = "./dataset/lab3-1/7_flower/validate"
test_dir = "./dataset/lab3-1/7_flower/test"

## Data preprocessing

Read training and validation dataset
- Data augmentation strategies using ImageDataGenerator

In [0]:
image_size = 224
num_class = 7

batch_size=32

train_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
 
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        val_dir,
        target_size=(224,224),
        batch_size=batch_size,
        class_mode='categorical')

FileNotFoundError: ignored

## `To do`

You are supposed to implement your own model using any techniques with the provided flower dataset with 7 classes as can described below.
    - Training dataset contains 336 images.
    - Validation dataset contains 112 images.
    - Testing dataset contains 112 images.
And you can measure the model performance by F1-score evaluator which its codes are also provided in the cell below.

## Evaluation

In [0]:
class_to_idx = train_generator.class_indices
idx_to_class = {v: k for k, v in class_to_idx.items()}

In [0]:
y_trues = []
y_preds = []
        
for class_idx in tqdm(range(num_class)):
    label = idx_to_class[class_idx]
    file_list = glob.glob("{}/{}/*.jpg".format(test_dir, label))
    for raw_image in file_list:
        inputShape = (224,224)
        image = load_img(raw_image, target_size=inputShape)
        image = img_to_array(image)
        image = preprocess_input(image)
        image = np.expand_dims(image, axis=0)
        
        # Change `model` to be your model varaible here
        preds = model.predict(image)
        pred_class = np.argmax(preds)

        y_trues.append(class_idx)
        y_preds.append(pred_class)

In [0]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print("-- Performace Model: (acc:{:.4f}, f1_micro:{:.4f}, f1_macro:{:.4f})".format(accuracy_score(y_trues, y_preds), 
                                                                                f1_score(y_trues, y_preds, average='micro'),
                                                                                f1_score(y_trues, y_preds, average='macro')))
